# Neural Network demo

Start a Mosquitto container first. For example:
- Use `codes\_demo\1_start_broker.sh` to start a Mosquitto container on Raspberry Pi.
- Config files are in `mqtt_config\mqtt`.
- set `allow_anonymous true` in `mqtt_config\mqtt\config\mosquitto.conf` to allow anonymous client.

## Getting Started
What this notebook does:  
- Using:
 - a client on PC
 - 6 ESP8266 modules (NodeMCU and D1 mini) as remote nodes
- List connected nodes
- Rename remote nodes
- Setup neural network configuration (connections, weights, thresholds)
- Fire up neurons and get logs.

In [1]:
import os
import sys
import time
 
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'client')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'node')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'shared')))
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, os.path.sep.join(['..', 'codes']), 'micropython')))
 
import client
from collections import OrderedDict

In [2]:
import pandas as pd
from pandas import DataFrame
from time import sleep
REFRACTORY_PERIOD = 0.1   # 0.1 seconds

### List of neurons

In [3]:
neurons = ['n_Alpha', 'n_Beta', 'n_Lambda'] 
# neurons = ['n_Alpha']
neurons

['n_Alpha', 'n_Beta', 'n_Lambda']

## Start client

In [4]:
the_client = client.Client()
the_client.start()

while not the_client.status['Is connected']:            
    time.sleep(1)
    print('Node not ready yet.')

My name is Client_366

Sending 277 bytes
Message:
OrderedDict([('command', 'set connection name'), ('correlation_id', '2017-06-10 11:07:13.381400'), ('kwargs', {'name': 'Client_366'}), ('message_id', '2017-06-10 11:07:13.381400'), ('message_type', 'command'), ('need_result', True), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


[Connected: ('123.110.13.5', 1883)]
[Listen to messages]
Node not ready yet.


## Utility functions

In [5]:
# # Ask Hub for a list of connected nodes
# def list_nodes():
#     the_client.node.worker.roll_call()
#     time.sleep(2)
#     remote_nodes = sorted(the_client.node.worker.contacts.keys())

#     print('\n[____________ Connected nodes ____________]\n')
#     print('\nConnected nodes:\n{}\n'.format(remote_nodes))
        
#     return remote_nodes

In [5]:
def reset_node(node):
    message = {'message_type': 'exec',
               'to_exec': 'import machine;machine.reset()'}
    the_client.request(node, message) 

In [6]:
def fire(node):
    message = {'message_type': 'function',
               'function': 'fire'}
    the_client.request(node, message) 

def addConnection(node, neuron):
    message = {'message_type': 'function',
               'function': 'addConnection',
               'kwargs': {'neuron_id': neuron}}
    the_client.request(node, message) 
    
def set_connections(node, connections):
    message = {'message_type': 'function',
               'function': 'setConnections',
               'kwargs': {'connections': connections}}
    the_client.request(node, message)     
    
def get_connections(node):
    message = {'message_type': 'function',
               'function': 'getConnections', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()    

def setWeight(node, neuron, weight):
    message = {'message_type': 'function',
               'function': 'setWeight',
               'kwargs': {'neuron_id': neuron,
                          'weight': weight,}}
    the_client.request(node, message) 

def setThreshold(node, threshold):
    message = {'message_type': 'function',
               'function': 'setThreshold',
               'kwargs': {'threshold': threshold}}
    the_client.request(node, message) 
        
def getConfig(node):
    message = {'message_type': 'function',
               'function': 'getConfig', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def getLog(node):
    message = {'message_type': 'function',
               'function': 'getLog', 
               'need_result': True}
    _, result = the_client.request(node, message) 
    return result.get()

def emptyLog(node):
    message = {'message_type': 'function',
               'function': 'emptyLog'}
    the_client.request(node, message)
    
def emptyLogs():
    for neuron in neurons:
        emptyLog(neuron)
        
def mergeLogs():
    logs = []
    
    for neuron in neurons:
        if neuron != the_client.node.worker.name:  # exclude client self
            currentLog = getLog(neuron)
            if currentLog:
                logs += currentLog 
            
    df = DataFrame(list(logs), columns = ['time', 'neuron', 'message']) 
    df.set_index('time', inplace = True)
    df.sort_index(inplace = True)
    
    return df        

In [7]:
def printConfig(neuron):
    print('{0:_^78}\n {1}\n'.format(neuron + " config:", getConfig(neuron)))

### Reset neurons

In [8]:
# reset_node('Hub');

### Probe neurons by blinking LEDs

In [9]:
messages = {}
messages['blink_led'] = {'message_type': 'command',
                         'command': 'blink led',
                         'kwargs': {'times': 3, 'on_seconds': 0.1, 'off_seconds': 0.1}}

the_client.request('Hub', messages['blink_led']);


Sending 275 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-06-10 11:07:27.012400'), ('kwargs', {'on_seconds': 0.1, 'off_seconds': 0.1, 'times': 3}), ('message_id', '2017-06-10 11:07:27.012400'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Data received: 275 bytes
Message:
OrderedDict([('command', 'blink led'), ('correlation_id', '2017-06-10 11:07:27.012400'), ('kwargs', {'times': 3, 'off_seconds': 0.1, 'on_seconds': 0.1}), ('message_id', '2017-06-10 11:07:27.012400'), ('message_type', 'command'), ('receiver', 'Hub'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



### Setup connections / weights / thresholds

In [10]:
addConnection('n_Alpha', 'n_Lambda');
addConnection('n_Beta', 'n_Lambda');

setWeight('n_Lambda', 'n_Alpha', 1); 
setWeight('n_Lambda', 'n_Beta', 1);


Sending 259 bytes
Message:
OrderedDict([('correlation_id', '2017-06-10 11:07:42.935400'), ('function', 'addConnection'), ('kwargs', {'neuron_id': 'n_Lambda'}), ('message_id', '2017-06-10 11:07:42.935400'), ('message_type', 'function'), ('receiver', 'n_Alpha'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 258 bytes
Sending 268 bytes

Message:
OrderedDict([('correlation_id', '2017-06-10 11:07:43.138400'), ('function', 'addConnection'), ('kwargs', {'neuron_id': 'n_Lambda'}), ('message_id', '2017-06-10 11:07:43.138400'), ('message_type', 'function'), ('receiver', 'n_Beta'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])
Message:
OrderedDict([('correlation_id', '2017-06-10 11:07:43.733400'), ('function', 'setWeight'), ('kwargs', {'neuron_id': 'n_Alpha', 'weight': 1}), ('message_id', '2017-06-10 11:07:43.733400'), ('message_type', 'function'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



Sending 267 bytes
Message:
OrderedDic

In [13]:
setThreshold('n_Lambda', 2.8);  # input enough to trigger Lambda
fire('n_Alpha');
fire('n_Beta');


Sending 252 bytes
Message:
OrderedDict([('correlation_id', '2017-06-10 11:08:19.053400'), ('function', 'setThreshold'), ('kwargs', {'threshold': 2.8}), ('message_id', '2017-06-10 11:08:19.053400'), ('message_type', 'function'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 213 bytes
Message:
OrderedDict([('correlation_id', '2017-06-10 11:08:19.307400'), ('function', 'fire'), ('message_id', '2017-06-10 11:08:19.307400'), ('message_type', 'function'), ('receiver', 'n_Alpha'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 212 bytes
Message:
OrderedDict([('correlation_id', '2017-06-10 11:08:20.648400'), ('function', 'fire'), ('message_id', '2017-06-10 11:08:20.648400'), ('message_type', 'function'), ('receiver', 'n_Beta'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [14]:
setThreshold('n_Lambda', 2.8);  # input not enough to trigger Lambda
fire('n_Alpha');
fire('n_Beta');


Sending 252 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:56.484000'), ('function', 'setThreshold'), ('kwargs', {'threshold': 2.8}), ('message_id', '2017-06-02 10:04:56.484000'), ('message_type', 'function'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 213 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:56.600000'), ('function', 'fire'), ('message_id', '2017-06-02 10:04:56.600000'), ('message_type', 'function'), ('receiver', 'n_Alpha'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])

Sending 212 bytes

Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:56.774000'), ('function', 'fire'), ('message_id', '2017-06-02 10:04:56.774000'), ('message_type', 'function'), ('receiver', 'n_Beta'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [13]:
setThreshold('n_Lambda', 1.8);  # input not enough to trigger Lambda
fire('n_Alpha'); 


Sending 252 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:54.317000'), ('function', 'setThreshold'), ('kwargs', {'threshold': 1.8}), ('message_id', '2017-06-02 10:04:54.317000'), ('message_type', 'function'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 213 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:54.608000'), ('function', 'fire'), ('message_id', '2017-06-02 10:04:54.608000'), ('message_type', 'function'), ('receiver', 'n_Alpha'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [15]:
setThreshold('n_Lambda', 1.8);  # input enough to trigger Lambda
fire('n_Alpha');
fire('n_Beta');


Sending 252 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:59.103000'), ('function', 'setThreshold'), ('kwargs', {'threshold': 1.8}), ('message_id', '2017-06-02 10:04:59.103000'), ('message_type', 'function'), ('receiver', 'n_Lambda'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 213 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:59.314000'), ('function', 'fire'), ('message_id', '2017-06-02 10:04:59.314000'), ('message_type', 'function'), ('receiver', 'n_Alpha'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])


Sending 212 bytes
Message:
OrderedDict([('correlation_id', '2017-06-02 10:04:59.390000'), ('function', 'fire'), ('message_id', '2017-06-02 10:04:59.390000'), ('message_type', 'function'), ('receiver', 'n_Beta'), ('reply_to', 'Client_366'), ('sender', 'Client_366')])



In [371]:
# setThreshold('n_Lambda', 1.8);

# emptyLogs()
# sleep(REFRACTORY_PERIOD)
# fire('n_Alpha')
# fire('n_Beta') 
# sleep(2)
# mergeLogs()

In [372]:
# for neuron in reversed(neurons): printConfig(neuron)

## Stop the demo

In [15]:
# Stopping
the_client.stop()
the_client = None
print ('\n[________________ Demo stopped ________________]\n')

[Closed: ('123.110.13.5', 1883)]
[________________ Demo stopped ________________]


